In [1]:
%load_ext autoreload
%autoreload 2

In [86]:
from dataset import PathMNIST

In [78]:
import torchvision.transforms as transforms
from constant import PathMNISTmeta

from torch.utils.data import DataLoader

In [79]:
train_transform = transforms.Compose([
    transforms.RandomCrop(28, padding=4, padding_mode="reflect"),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=PathMNISTmeta.MEAN, std=PathMNISTmeta.STD)
])

In [80]:
train_dataset = PathMNIST(
    split="train", download=False, 
    transform=train_transform,
    root="../data/medmnist2d/"
)

In [87]:
train_loader = DataLoader(train_dataset, batch_size=4, collate_fn=PathMNIST.collate_fn)

In [88]:
for x in train_loader:
    print(x)
    break

(tensor([[[[ 1.0154,  1.0154,  0.9835,  ...,  0.9517,  1.0154,  0.9517],
          [ 0.6016,  0.6334,  0.5061,  ...,  1.0154,  1.0154,  1.0154],
          [ 0.1241,  0.2833,  0.3469,  ...,  1.0154,  0.9517,  1.0154],
          ...,
          [ 1.0790,  1.0154,  1.0790,  ...,  0.8880,  0.9835,  0.8880],
          [ 1.0472,  1.0472,  1.0790,  ...,  0.0923,  0.2196,  0.0923],
          [ 0.9517,  0.9517,  0.9835,  ...,  1.0154,  0.9835,  1.0154]],

         [[ 1.5952,  1.5952,  1.5729,  ...,  1.5506,  1.5952,  1.5506],
          [ 1.2611,  1.2388,  1.1274,  ...,  1.5952,  1.6174,  1.5952],
          [ 0.8156,  0.9270,  0.9493,  ...,  1.5952,  1.5729,  1.5952],
          ...,
          [ 1.6397,  1.6174,  1.5952,  ...,  1.5061,  1.5506,  1.5061],
          [ 1.6174,  1.5952,  1.6397,  ...,  0.7265,  0.7934,  0.7265],
          [ 1.5284,  1.5506,  1.5952,  ...,  1.5729,  1.5729,  1.5729]],

         [[ 1.4781,  1.4781,  1.4781,  ...,  1.4148,  1.4781,  1.4148],
          [ 1.0668,  1.0668, 

In [91]:
x[0].shape

torch.Size([4, 3, 28, 28])

In [93]:
len(train_loader)

22499

In [72]:
torch.Tensor([1,2,3])

tensor([1, 2, 3])

In [59]:
x[1]

[array(0), array(4), array(7), array(5)]

In [28]:
x[1].shape

torch.Size([4, 1])

In [29]:
x[1].squeeze()

tensor([0, 4, 7, 5])

In [31]:
x[1].shape

torch.Size([4, 1])

In [33]:
x[1].squeeze(-1)

tensor([0, 4, 7, 5])

In [37]:
y = x[1].squeeze()

In [39]:
from torchvision.models import resnet18

In [60]:
from torch.utils.data import default_collate

In [63]:
default_collate

<function torch.utils.data._utils.collate.default_collate(batch)>